In [1]:
import requests
from bs4 import BeautifulSoup

In [42]:
import requests
from bs4 import BeautifulSoup

# Define the URL to crawl
url = "https://lightning.ai/docs/pytorch/latest/starter/introduction.html"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the div element with class "rst-content"
    div_content = soup.find('div', class_='rst-content')

    if div_content:
        # Find all section tags within the div_content
        sections = div_content.find_all('section')

        for section in sections[1:]:
            # Extract and print the content of each section
            section_content = section.get_text()
            print(section_content)
            print("=" * 40)  # Add a separator between sections

    else:
        print("Div element with class 'rst-content' not found. Check the HTML structure of the page.")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")



1: Install PyTorch Lightning¶

For pip users
pip install lightning



For conda users
conda install lightning -c conda-forge



Or read the advanced install guide



2: Define a LightningModule¶
A LightningModule enables your PyTorch nn.Module to play together in complex ways inside the training_step (there is also an optional validation_step and test_step).
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L

# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


# define the LightningModule
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # tra

In [111]:
from urllib.parse import urlparse, urlunparse

In [112]:
def remove_fragement(url):
    parsed_url = urlparse(url)

    # Reconstruct the URL without the fragment identifier
    new_url = urlunparse(parsed_url._replace(fragment=''))
    return new_url


In [118]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

crawled_urls = set()
# Function to fetch and extract links from a page
def get_links(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            links = []
            for a_tag in soup.find_all('a', href=True):
                link = a_tag['href']
                links.append(link)
            return links
    except Exception as e:
        print(f"Failed to fetch links from {url}: {e}")
    return []

# Function to recursively fetch links within the same domain
def fetch_links_recursive(base_url, current_url, visited_urls, max_depth=4):
    if current_url in visited_urls or max_depth == 0:
        return

    visited_urls.add(current_url)
    links = get_links(current_url)
    for link in links:
        absolute_url = urljoin(current_url, link)
        parsed_url = urlparse(absolute_url)
        if parsed_url.netloc == base_url.netloc:
            crawled_urls.add(remove_fragement(absolute_url))
            fetch_links_recursive(base_url, absolute_url, visited_urls, max_depth - 1)


In [119]:
base_url = urlparse("https://lightning.ai/docs/pytorch/latest/starter/introduction.html")
visited_urls = set()
fetch_links_recursive(base_url, base_url.geturl(), visited_urls)

In [120]:
# rm crawled/*

In [121]:
len(crawled_urls)

275

In [122]:
crawled_urls

{'https://lightning.ai/docs/fabric/',
 'https://lightning.ai/docs/pytorch/latest/_images/custom_loop.png',
 'https://lightning.ai/docs/pytorch/latest/_images/ddp.gif',
 'https://lightning.ai/docs/pytorch/latest/_modules/lightning/fabric/utilities/throughput.html',
 'https://lightning.ai/docs/pytorch/latest/_modules/lightning/pytorch/core/module.html',
 'https://lightning.ai/docs/pytorch/latest/_modules/lightning/pytorch/trainer/trainer.html',
 'https://lightning.ai/docs/pytorch/latest/_sources/accelerators/gpu.rst.txt',
 'https://lightning.ai/docs/pytorch/latest/_sources/accelerators/tpu.rst.txt',
 'https://lightning.ai/docs/pytorch/latest/_sources/advanced/speed.rst.txt',
 'https://lightning.ai/docs/pytorch/latest/_sources/api_references.rst.txt',
 'https://lightning.ai/docs/pytorch/latest/_sources/common/checkpointing.rst.txt',
 'https://lightning.ai/docs/pytorch/latest/_sources/common/index.rst.txt',
 'https://lightning.ai/docs/pytorch/latest/_sources/common/lightning_module.rst.txt

In [123]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_sections_to_csv(url, output_file):
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the div element with class "rst-content"
        div_content = soup.find('div', class_='rst-content')

        if div_content:
            # Find all section tags within the div_content
            sections = div_content.find_all('section')

            # Create a list to store the sections
            section_contents = []

            for section in sections[1:]:
                # Extract the content of each section
                section_content = section.get_text()
                section_contents.append(section_content)

            # Create a DataFrame with URL and Section Content columns
            df = pd.DataFrame({"URL": [url] * len(section_contents), "Section Content": section_contents})

            # Save the DataFrame to a CSV file
            df.to_csv(output_file, index=False)

        else:
            print("Div element with class 'rst-content' not found. Check the HTML structure of the page.")

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

# Example usage
url = "https://lightning.ai/docs/pytorch/latest"
output_file = "sections.csv"
extract_sections_to_csv(url, output_file)

In [124]:
from rich.progress import track

for i, url in enumerate(track(crawled_urls)):
    output_file = f"crawled/{i}.csv"
    extract_sections_to_csv(url, output_file)

Output()

Div element with class 'rst-content' not found. Check the HTML structure of the page.

Div element with class 'rst-content' not found. Check the HTML structure of the page.

Div element with class 'rst-content' not found. Check the HTML structure of the page.

Div element with class 'rst-content' not found. Check the HTML structure of the page.

...

Div element with class 'rst-content' not found. Check the HTML structure of the page.

Div element with class 'rst-content' not found. Check the HTML structure of the page.

Div element with class 'rst-content' not found. Check the HTML structure of the page.

Div element with class 'rst-content' not found. Check the HTML structure of the page.

Div element with class 'rst-content' not found. Check the HTML structure of the page.